# data_cleaning_binary

## By: Patrick Cavins

### What needs to be done? 

After bringing in some initial images using the PubChem PUGRest API a coulple data munging concerns came up. 

- Nonstandard Isotopes 
- Two imgages in a single output file

In [2]:
# Import 

import pandas as pd
import requests
import os
import time
import regex as re 

In [3]:
file_name = '2_methylbutyl_cyclopentene_134887291.csv'

In [4]:
df = pd.read_csv(f'./data/{file_name}')

df.drop(columns='Unnamed: 0', inplace=True)

In [5]:
df.head()

,cid,ring,image_saved_as
0,19540,1,2_methylbutyl_cyclopentene_19540
1,16270,1,2_methylbutyl_cyclopentene_16270
2,16269,1,2_methylbutyl_cyclopentene_16269
3,137211,1,2_methylbutyl_cyclopentene_137211
4,77414,1,2_methylbutyl_cyclopentene_77414


In [6]:
df.shape

(189, 3)

In [7]:
iupac = []
mf = []

for i in df['cid']:
    
    #Print
    print (i)
    
    #constructing the url for the specific CID 
    entry = i
    url_iupac = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{entry}/property/IUPACName/TXT'
    url_MF = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{entry}/property/MolecularFormula/TXT'
    
    #calling the PUG Rest PubChem API
    api_call_iupac = requests.get(url_iupac)
    api_call_MF = requests.get(url_MF)
    
    #Saving the output to a variable
    iupac_list = api_call_iupac.text
    MF_list = api_call_MF.text
    
    iupac.append(iupac_list)
    mf.append(MF_list)
    
        
    time.sleep(0.2)

#adding variables to the data frame   
df['iupac'] = iupac
df['mf'] = mf  
# df['canonical_smiles'] = smiles 

19540
16270
16269
137211
77414
521217
517741
138541
138257
137755
137210
19893859
6427086
530396
523528
523527
521236
54101622
20837636
19893766
19893555
14764015
6537508
6431020
6427085
544055
523530
523529
523489
521449
520404
520128
123224949
91199232
90713669
60007293
59994267
59954505
59954503
59954496
59954495
59954478
59954469
59954454
59954450
59954445
59954414
59936799
59911458
59911321
59746976
59117969
59066598
59039354
59039321
58688977
58639765
58504179
57285679
57198894
57162921
57125758
57115232
57045719
54526457
54252328
53678344
23384891
23384890
23267477
22212143
22129074
22129047
19893868
19893808
19893667
19893613
19893595
19893531
18730047
18730042
523526
129777607
129716583
124007590
123984630
123968381
123950095
123943353
123942691
123913323
123866552
123857423
123853415
123833176
123811136
123786274
123781570
123777478
123726806
123691427
123663745
123660410
123625055
123561961
123543629
123505979
123455735
123434097
123427642
123376011
123372890
123356282
12335

In [8]:
df.shape

(189, 5)

In [55]:
df.to_csv(f'./data/updated/updated_{file_name}', index=False)

#### So this is FALSE 
```pattern = r';'
string = '3-methylidenepent-1-ene'
print (re.search(pattern=pattern, string=string))```

#### So this is TRUE
```pattern = r';'
string = 'magnesium;pent-1-ene;bromide'
print (re.search(pattern=pattern, string=string))```

In [56]:
#Start Here... 

In [57]:
df = pd.read_csv(f'./data/updated/updated_{file_name}')

In [58]:
df['mf']=df['mf'].map(lambda x: x.replace('\n', ''))
df['iupac']=df['iupac'].map(lambda x: x.replace('\n', ''))

df.head()

,cid,ring,image_saved_as,iupac,mf
0,19540,1,2_methylbutyl_cyclopentene_19540,pentylcyclopentane,C10H20
1,16270,1,2_methylbutyl_cyclopentene_16270,propylcyclopentane,C8H16
2,16269,1,2_methylbutyl_cyclopentene_16269,butylcyclopentane,C9H18
3,137211,1,2_methylbutyl_cyclopentene_137211,decylcyclopentane,C15H30
4,77414,1,2_methylbutyl_cyclopentene_77414,2-methylpropylcyclopentane,C9H18


In [59]:
df.shape

(189, 5)

In [60]:
# pattern = r';'
# string = '3-methylidenepent-1-ene'
# bool(re.findall(pattern=pattern, string=string))

# pattern = r';'
# string = 'hex-1-ene;zinc;iodide'
# bool(re.findall(pattern=pattern, string=string))

In [61]:
#These are index values which we will want to eventually drop 
index_val = []

#Regex Pattern
pattern = r';'

#Searching to Evaluate for the IUPAC Names which contain ';' = False Positive Images
for index, row in df.iterrows():

    #using regex to find the false_positives
    if bool(re.findall(pattern=pattern, string=row[3]))==1:
        index_val.append(index)

#Identify the index values for the rows that we want to keep
indexes_to_keep =set(range(df.shape[0])) - set(index_val)
df_sliced = df.take(list(indexes_to_keep))

#Building the dataframe of false_positives
false_positives = pd.DataFrame.from_dict(dict(df.iloc[index_val]))


#Save to CSV
df_sliced.to_csv(f'./data/cleaned/cleaned_{file_name}', index=False)
false_positives.to_csv(f'./data/false_positives/false_positives_{file_name}', index=False)
    

#Print Out index list created from the for loop   
print (index_val)
print ('-'*50)
print (len(index_val))


# Inspiration, 1 = 'https://www.pythonprogramming.in/pandas-examples/how-to-append-rows-in-a-pandas-dataframe-using-a-for-loop.html'
# Inspiration, 2 = https://stackoverflow.com/questions/37727018/pandas-dataframe-move-rows-from-one-dataframe-to-another?rq=1
# Inspiration, 3 = https://stackoverflow.com/questions/14661701/how-to-drop-a-list-of-rows-from-pandas-dataframe

[33, 34, 172]
----------------------------------------------------------------------------------------------------
3


In [62]:
df_sliced.head()

print (df_sliced.shape) 

(186, 5)


In [63]:
false_positives.head()
print (false_positives.shape)

(3, 5)


In [64]:
df_sliced.head()

,cid,ring,image_saved_as,iupac,mf
0,19540,1,2_methylbutyl_cyclopentene_19540,pentylcyclopentane,C10H20
1,16270,1,2_methylbutyl_cyclopentene_16270,propylcyclopentane,C8H16
2,16269,1,2_methylbutyl_cyclopentene_16269,butylcyclopentane,C9H18
3,137211,1,2_methylbutyl_cyclopentene_137211,decylcyclopentane,C15H30
4,77414,1,2_methylbutyl_cyclopentene_77414,2-methylpropylcyclopentane,C9H18


In [65]:
false_positives.head()

,cid,ring,image_saved_as,iupac,mf
33,91199232,1,2_methylbutyl_cyclopentene_91199232,"1,3-diethylcyclopentane;ethane",C13H30
34,90713669,1,2_methylbutyl_cyclopentene_90713669,"1,3-diethylcyclopentane;ethane",C11H24
172,22129134,1,2_methylbutyl_cyclopentene_22129134,lithium;2-methylpropylcyclopentane,C9H17Li
